In [32]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *
from option_slam_earnings import *
from reuters_query import reuters_query, reuters_insiders
from alphaquery import alphaquery

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
from yahoo_earnings import *
from lookup_earnings import lookup_earnings

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Processing')
from hv_calc import *

os.chdir(main_dir)

dbs_dir = 'D:\\Price Data'

os.chdir(dbs_dir)

price_engine = create_engine('sqlite:///histprices.db', echo=False)

returns_engine = create_engine('sqlite:///returnProfiles.db', echo=False)

inspector = inspect(price_engine)
return_inspector = inspect(returns_engine)

In [2]:
spx = yahoo_query('^GSPC', dt.datetime(1999,1,1))
spx.hist_prices_query()
spx = spx.hist_prices

In [15]:
names_query = 'SELECT DISTINCT Underlying FROM {}'.format(inspector.get_table_names()[0])

stock_names = pd.read_sql_query(names_query, con = price_engine, parse_dates = True)

In [124]:
import time

start_time = time.time()

single_names = []

for ticker in stock_names.Underlying:
#     curr_early_query = 'SELECT Date, Underlying, adjclose FROM {0} WHERE Underlying = "{1}" LIMIT 1'.format(inspector.get_table_names()[0], ticker)
#     curr_last_query = 'SELECT Date, Underlying, adjclose FROM {0} WHERE Underlying = "{1}" ORDER BY Date DESC LIMIT 1'.format(inspector.get_table_names()[0], ticker)
#     curr_stock = pd.read_sql_query(curr_early_query, con = price_engine, index_col = 'Date', parse_dates = ['Date'])
#     curr_stock_last = pd.read_sql_query(curr_last_query, con = price_engine, index_col = 'Date', parse_dates = ['Date'])
    curr_query = 'SELECT Date, Underlying, adjclose FROM {0} WHERE Underlying = "{1}"'.format(inspector.get_table_names()[0], ticker)
    curr_stock = pd.read_sql_query(curr_query, con = price_engine, index_col = 'Date', parse_dates = ['Date'])
    curr_stock = curr_stock.head(1).reset_index().join(curr_stock.tail(1).reset_index(), lsuffix = '_start').set_index('Underlying')
    del curr_stock['Underlying_start']
    curr_stock.columns = ['Start_Date', 'Start_Price', 'End_Date', 'End_Price']
    single_names.append(curr_stock)
print('Completed in {} seconds'.format(time.time() - start_time))

Completed in 35276.8602437973 seconds


In [161]:
stock_returns = pd.concat(single_names, axis = 0)
spy_period = stock_returns[stock_returns.index == 'SPY']
spy_period['TotalYears'] = (spy_period.End_Date - spy_period.Start_Date).dt.days/365.25
spy_period['CAGR'] = (spy_period.End_Price/spy_period.Start_Price)**(1/spy_period.TotalYears) - 1
stock_returns = stock_returns.join(returns.iloc[:,-2:]).dropna().drop_duplicates()
stock_returns['TotalYears'] = (stock_returns.End_Date - stock_returns.Start_Date).dt.days/365.25
stock_returns['CAGR'] = (stock_returns.End_Price/stock_returns.Start_Price)**(1/stock_returns.TotalYears) - 1
stock_returns = stock_returns.dropna().sort_values('CAGR', ascending = False)

In [183]:
beat_spy = stock_returns[stock_returns.CAGR > spy_period.loc['SPY','CAGR']]
lost_spy = stock_returns[stock_returns.CAGR <= spy_period.loc['SPY','CAGR']]

len(beat_spy)/len(stock_returns)

0.44821283509341997

In [192]:
(beat_spy[['sector','industry']].groupby('sector').count()/len(beat_spy)).sort_values('industry', ascending = False)
(beat_spy[['sector','industry','End_Price']].groupby(['sector','industry']).count()/len(beat_spy)).sort_values('End_Price', ascending = False).head(10)

End_Price
sector             industry                                  
Financial Services Banks - Regional - US               0.0829
                   Asset Management                    0.0585
Healthcare         Biotechnology                       0.0444
Technology         Software - Application              0.0376
Industrials        Diversified Industrials             0.0263
                   Business Services                   0.0217
Energy             Oil & Gas E&P                       0.0213
Healthcare         Diagnostics & Research              0.0168
Technology         Software - Infrastructure           0.0168
Financial Services Insurance - Property & Casualty     0.0150

In [193]:
(lost_spy[['sector','industry','End_Price']].groupby(['sector','industry']).count()/len(lost_spy)).sort_values('End_Price', ascending = False).head(10)

End_Price
sector                 industry                                           
Healthcare             Biotechnology                                0.1318
Financial Services     Asset Management                             0.0880
                       Banks - Regional - US                        0.0519
Energy                 Oil & Gas E&P                                0.0283
Technology             Software - Application                       0.0276
                       Communication Equipment                      0.0232
Energy                 Oil & Gas Equipment & Services               0.0184
Communication Services Telecom Services                             0.0169
Industrials            Business Services                            0.0166
Healthcare             Drug Manufacturers - Specialty & Generic     0.0166

In [195]:
stock_returns[stock_returns['industry'] == 'Software - Application']

,Start_Date,Start_Price,End_Date,End_Price,industry,sector,TotalYears,CAGR
Underlying,,,,,,,,
LN,2015-10-12,0.0500,2018-12-28,34.0300,Software - Application,Technology,3.2115,6.6227
MDB,2017-10-19,32.0700,2018-12-28,80.5000,Software - Application,Technology,1.1910,1.1657
AYX,2017-03-24,15.5000,2018-12-28,58.4000,Software - Application,Technology,1.7632,1.1219
SSNT,2015-01-02,0.2294,2018-12-28,2.5300,Software - Application,Technology,3.9863,0.8261
TTD,2016-09-21,30.1000,2018-12-28,114.4500,Software - Application,Technology,2.2669,0.8025
SHOP,2015-05-20,17.0000,2018-12-28,133.4600,Software - Application,Technology,3.6085,0.7701
EVBG,2016-09-16,15.2500,2018-12-28,54.7100,Software - Application,Technology,2.2806,0.7509
TEAM,2015-12-09,21.0000,2018-12-28,87.6000,Software - Application,Technology,3.0527,0.5966
SQ,2015-11-19,13.0700,2018-12-28,55.8700,Software - Application,Technology,3.1075,0.5960


In [199]:
long_term = stock_returns[stock_returns.TotalYears > 15]

beat_spy = long_term[long_term.CAGR > spy_period.loc['SPY','CAGR']]
lost_spy = long_term[long_term.CAGR <= spy_period.loc['SPY','CAGR']]

len(beat_spy)/len(stock_returns)

0.27619821283509344

In [202]:
(beat_spy[['sector','industry']].groupby('sector').count()/len(beat_spy)).sort_values('industry', ascending = False)

,industry
sector,
Financial Services,0.2353
Industrials,0.1500
Consumer Cyclical,0.1206
Technology,0.1110
Healthcare,0.0824
Basic Materials,0.0625
Real Estate,0.0618
Energy,0.0603
Consumer Defensive,0.0551


In [203]:
(beat_spy[['sector','industry','End_Price']].groupby(['sector','industry']).count()/len(beat_spy)).sort_values('End_Price', ascending = False).head(10)

End_Price
sector             industry                                  
Financial Services Banks - Regional - US               0.0912
                   Asset Management                    0.0625
Industrials        Diversified Industrials             0.0324
Energy             Oil & Gas E&P                       0.0257
Technology         Software - Application              0.0221
Industrials        Business Services                   0.0199
Healthcare         Biotechnology                       0.0184
Industrials        Aerospace & Defense                 0.0169
Financial Services Insurance - Property & Casualty     0.0169
Basic Materials    Specialty Chemicals                 0.0162

In [210]:
beat_spy[(beat_spy.sector == 'Technology') & 
         (beat_spy.End_Price > 20)]

,Start_Date,Start_Price,End_Date,End_Price,industry,sector,TotalYears,CAGR
Underlying,,,,,,,,
AAPL,1999-01-04,0.9866,2018-12-28,156.2300,Consumer Electronics,Technology,19.9808,0.2885
NTES,2000-06-30,2.7129,2018-12-28,236.9100,Internet Content & Information,Technology,18.4942,0.2734
CTSH,1999-01-04,0.6128,2018-12-28,62.9300,Information Technology Services,Technology,19.9808,0.2609
NVDA,1999-01-22,1.5169,2018-12-28,133.6500,Semiconductors,Technology,19.9316,0.2519
BMI,1999-01-04,0.6911,2018-12-28,48.0700,Scientific & Technical Instruments,Technology,19.9808,0.2365
MLAB,1999-01-04,3.4233,2018-12-28,207.2900,Scientific & Technical Instruments,Technology,19.9808,0.2280
ATVI,1999-01-04,0.8241,2018-12-28,46.8000,Electronic Gaming & Multimedia,Technology,19.9808,0.2240
AMOT,1999-01-04,0.8255,2018-12-28,44.1900,Electronic Components,Technology,19.9808,0.2204
ANSS,1999-01-04,2.7344,2018-12-28,142.9900,Software - Application,Technology,19.9808,0.2190


In [206]:
lost_spy[lost_spy.sector == 'Technology']

,Start_Date,Start_Price,End_Date,End_Price,industry,sector,TotalYears,CAGR
Underlying,,,,,,,,
DXC,1999-01-04,18.3432,2018-12-28,52.9500,Information Technology Services,Technology,19.9808,0.0545
VSH,1999-01-04,6.1953,2018-12-28,17.8700,Semiconductors,Technology,19.9808,0.0544
DSGX,1999-01-27,9.2500,2018-12-28,26.2600,Software - Application,Technology,19.9179,0.0538
SINA,2000-04-13,20.6875,2018-12-28,54.5800,Internet Content & Information,Technology,18.7077,0.0532
ORBK,1999-01-04,19.8889,2018-12-28,55.7800,Scientific & Technical Instruments,Technology,19.9808,0.0530
ARW,1999-01-04,24.5625,2018-12-28,68.2400,Electronics Distribution,Technology,19.9808,0.0525
ENTG,2000-07-11,11.0631,2018-12-28,27.4700,Semiconductor Equipment & Materials,Technology,18.4641,0.0505
GLW,1999-01-04,11.5822,2018-12-28,29.7500,Electronic Components,Technology,19.9808,0.0483
AVX,1999-01-04,5.8727,2018-12-28,15.0700,Electronic Components,Technology,19.9808,0.0483


In [ ]:
returns_query = 'SELECT * FROM annualReturns'
returns = pd.read_sql_query(returns_query, con = returns_engine).drop_duplicates().set_index('Underlying')

In [ ]:
yearly_dict = {'Percent_Pos_returns':[],
               'Percent_Beat_spy': [],
               'Percent_Neg_returns':[],
               'Percent_Lost_spy':[]}

for year in returns.columns.tolist()[:-2]:
    curr_annual_df = returns[[year] + returns.columns.tolist()[-2:]].dropna(subset = [year])
    try:
        curr_annual_df = curr_annual_df[curr_annual_df[year].str.isnumeric().isnull()]
    except:
        None
    curr_spy = curr_annual_df.loc['SPY',year]
    positives = curr_annual_df[curr_annual_df[year] > 0].dropna(subset = ['sector']).sort_values(year,
                                                                                                      ascending = False)
    gainers = curr_annual_df[curr_annual_df[year] > curr_spy].dropna(subset = ['sector']).sort_values(year,
                                                                                                      ascending = False)
    losers = curr_annual_df[curr_annual_df[year] <= curr_spy].dropna(subset = ['sector']).sort_values(year)
    negatives = curr_annual_df[curr_annual_df[year] <= 0].dropna(subset = ['sector']).sort_values(year)
    
    total_names = len(gainers) + len(losers)
    
    yearly_dict['Percent_Pos_returns'].append(round(len(positives)/total_names,4)*100)
    yearly_dict['Percent_Beat_spy'].append(round(len(gainers)/total_names,4)*100)
    yearly_dict['Percent_Neg_returns'].append(round(len(negatives)/total_names,4)*100)
    yearly_dict['Percent_Lost_spy'].append(round(len(losers)/total_names,4)*100)
    print(year)
    print('Percent Gainers: {}%'.format(str(round(len(gainers)/total_names,4)*100)))
    print('Percent Losers: {}%'.format(str(round(len(losers)/total_names,4)*100)))
    print('Percent Negatives: {}%'.format(str(round(len(negatives)/total_names,4)*100)))
    print('\n')

In [137]:
# pd.DataFrame(yearly_dict, index = returns.columns.tolist()[:-2])

In [ ]:
start_time = time.time()
i = 0
total_length = len(us_names)
return_engine = create_engine('sqlite:///returnProfiles.db', echo = False)

for ticker in us_names:
    annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols = return_profiles(ticker)
    if type(annual_returns) != str and ticker != 'AAPL':
        annual_returns.to_sql('annualReturns', con=return_engine, 
                              if_exists='append', index_label = 'Underlying')
        annualDDs.to_sql('annualDDs', con=return_engine, 
                         if_exists='append', index_label = 'Underlying')
        annualDownsides.to_sql('annualDownsides', con=return_engine, 
                               if_exists='append', index_label = 'Underlying')
        annualVols.to_sql('annualVols', con=return_engine, 
                          if_exists='append', index_label = 'Underlying')
        
        quarterly_returns.to_sql('quarterlyReturns', con=return_engine, 
                                 if_exists='append', index_label = 'Underlying')
        quarterlyDDs.to_sql('quarterlyDDs', con=return_engine, 
                            if_exists='append', index_label = 'Underlying')
        quarterlyDownsides.to_sql('quarterlyDownsides', con=return_engine, 
                                  if_exists='append', index_label = 'Underlying')
        quarterlyVols.to_sql('quarterlyVols', con=return_engine, 
                             if_exists='append', index_label = 'Underlying')    
    i += 1
    print('{0:.2f}% Completed'.format(i/total_length*100))

print("Completed in %s seconds" % (time.time() - start_time))

In [370]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
soup = bs(requests.get(wiki_url,'lxml').text,'lxml')
sp_const = soup.select('table[class="wikitable sortable"]')[0]
changes = soup.select('table[class="wikitable sortable"]')[1]

sp_dict = {}
sp_headers = []
for row in sp_const.find_all('th'):
    sp_headers.append(row.text.strip())
    sp_dict[row.text.strip()] = []
    
for row in sp_const.find_all('tr'):
    for i, col in enumerate(row.find_all('td')):
        col_header = sp_headers[i]
        curr_val = col.text.strip()
        if 'Date' in col_header:
            try:
                curr_val = dt.datetime.strptime(curr_val, '%Y-%m-%d')
            except:
                curr_val = np.nan
        sp_dict[col_header].append(curr_val)

sp_const = pd.DataFrame(sp_dict).set_index('Symbol')
sp_const = sp_const[['Security','Date first added','Founded','GICS Sector','GICS Sub Industry']]

changes_headers = ['Date','Ticker_Added','Security_Added',
                   'Ticker_Removed']

changes_dict = {k:[] for k in changes_headers}


rowspan = 0
for row in changes.find_all('tr'):
       
    for i, col in enumerate(row.find_all('td')[:-1]):
        curr_val = col.text.strip()
        if len(row) == 12:
            if i == 0:
                curr_val = dt.datetime.strptime(curr_val, '%B %d, %Y')
                curr_date = curr_val
            if i == 4:
                continue
            changes_dict[changes_headers[i]].append(curr_val)
            
        else:
            if i == 0:
                changes_dict[changes_headers[i]].append(curr_date)
                changes_dict[changes_headers[1]].append(curr_val)
            elif i == 3:
                continue
            else:
                changes_dict[changes_headers[i + 1]].append(curr_val)
changes = pd.DataFrame(changes_dict)    

In [433]:
sp_500_hist = {dt.datetime.today().date():sp_const}

curr_const = sp_const.copy()

for change_date in changes.Date.drop_duplicates():
    curr_date = change_date.date()
    curr_changes = changes[changes.Date == change_date]
    curr_drops = curr_changes.Ticker_Added.tolist()
    curr_adds = curr_changes.Ticker_Removed.tolist()
    
    curr_adds = list(filter(lambda x: x != '', curr_adds))
    curr_drops = list(filter(lambda x: x != '', curr_drops))
    
    curr_rows = pd.DataFrame({'Symbol': curr_adds}).set_index('Symbol')
    
    curr_const = curr_const[curr_const.index.isin(curr_drops) == False]
    curr_const = pd.concat([curr_const, curr_rows], axis = 0)
    
    sp_500_hist[curr_date] = curr_const
    

In [444]:
all_sp500_names = pd.concat(list(sp_500_hist.values())).reset_index()[['Symbol','Security']].drop_duplicates()